In [1]:
# import folium
# import json
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup
import webbrowser

# from html_table_parser import parser_functions as parser

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc

import platform

plt.rcParams['axes.unicode_minus']= False
if platform.system()== 'Windows':
    path= 'c:/Windows/Fonts/malgun.ttf'
    font_name= font_manager.FontProperties(fname=path).get_name()
    rc('font', family= font_name)
elif platform.system()=='Darwin':
    rc('font', family= 'AppleGothic')
else:
    print('포트를 지원하지 않습니다.')


In [43]:
# 좌표-> 맵에 표시하기
# gps = [35.172905, 129.130757]
# zoom = 18
# map_obj= folium.Map(location=gps, zoom_start=zoom)
# map_obj

# zoom=16
# stamen= folium.Map(location=gps, zoom_start=zoom, tiles='Stamen Toner')  # tiles='Stamen Terrain'
# stamen

In [45]:
# 주소 -> 좌표구하기, 좌표-> 주소구하기
# https://geopy.readthedocs.io/en/stable/

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.geocode("175 5th Avenue NYC")
print(location.address)
print((location.latitude, location.longitude))

Flatiron Building, 175, 5th Avenue, Flatiron District, Manhattan, Manhattan Community Board 5, New York County, NYC, New York, 10010, USA
(40.7410861, -73.9896298241625)


In [48]:
location = geolocator.reverse("37.59485258, 126.9419296")
print(location.address)

통일로48길, 홍은동, 남가좌동, 서대문구, 서울특별시, 03650, 대한민국


In [38]:
from geopy.distance import geodesic
newport_ri = (41.49008, -71.312796)
cleveland_oh = (41.499498, -81.695391)
print(geodesic(newport_ri, cleveland_oh).km)
#print(great_circle(newport_ri, cleveland_oh).miles)

866.4554329098688


In [39]:
import pandas as pd
df = pd.DataFrame({'name': ['paris', 'berlin', 'london']})

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df['location'] = df['name'].apply(geocode)

#df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [42]:
df

,name,location
0,paris,"(Paris, Île-de-France, France métropolitaine, ..."
1,berlin,"(Berlin, 10117, Deutschland, (52.5170365, 13.3..."
2,london,"(London, Greater London, England, SW1A 2DU, UK..."


In [ ]:
#공공데이터 활용하여 실거래가 가져오기

In [2]:
# https://www.data.go.kr/dataset/3050988/openapi.do
# https://tariat.tistory.com/101 참고, [부동산 API1] 공공데이터 활용하여 전월세 실거래가 가져오기

#http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade #아파트매매 실거래
#http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptRent # 아파트 전월세

def collect_AptTrade(ym,lawd_cd):

    API_KEY = 'QnwgLA6Jo7nNz7QqCikCRpTpDrNLEv3HPVs0Oj1x4XP3D4Xmzp2weptLAgNsQ9dRqZKmtorwob9hrzul36Jn8A%3D%3D'
    url="http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade"

    # &numOfRows=1000"+
    url=url+"?&LAWD_CD=" + lawd_cd + "&DEAL_YMD=" + ym + "&serviceKey=" + API_KEY

    # webbrowser.open(url)

    resultXML = urlopen(url)
    result = resultXML.read()
    xmlsoup = BeautifulSoup(result, 'lxml-xml')

    te=xmlsoup.findAll("item")

    sil=pd.DataFrame()


    for t in te:
        real_price=t.find("거래금액").text
        build_y=t.find("건축년도").text
        year=t.find("년").text
        month=t.find("월").text
        day=t.find("일").text
        dong=t.find("법정동").text
        apt_nm=t.find("아파트").text
        size=t.find("전용면적").text

        try:
            jibun=t.find("지번").text
        except:
            jibun=""

        ji_code=t.find("지역코드").text
        floor=t.find("층").text

        temp = pd.DataFrame(([[build_y,year,month,day,dong,apt_nm,size,jibun,ji_code,floor,real_price]]), 
                            columns=["build_y","year","month","day","dong","apt_nm","size","jibun","ji_code","floor","real_price"])
        sil=pd.concat([sil,temp])

    sil=sil.reset_index(drop=True)

    return sil

In [4]:
gu_list= ['26110', '26140', '26170', '26200', '26230', '26260', '26290',
       '26320', '26350', '26380', '26410', '26440', '26470', '26500',
       '26530', '26710', '11110', '11140', '11170', '11200', '11215',
       '11230', '11260', '11290', '11305', '11320', '11350', '11380',
       '11410', '11440', '11470', '11500', '11530', '11545', '11560',
       '11590', '11620', '11650', '11680', '11710', '11740'] # 41개의 지역구

ym_list= ['201701','201702','201703','201704','201705','201706','201707','201708','201709','201710','201711','201712',
          '201801','201802','201803','201804','201805','201806','201807','201808','201809','201810'] # 22개월

# gu_list= ['11680', '11110', '11500', '11350', '11170', '11650', '11440','26140','26710'] 
# ym_list= ['200912','201304','201309','201312','201401','201411','201503','201510','201603','201606','201810']


sil_trade=pd.DataFrame()

for m in ym_list:
    for co in gu_list:
        temp= collect_AptTrade(m,co)
        sil_trade= pd.concat([sil_trade,temp])
        print(co+", "+str(len(temp))+" is completed")

    print("*"+str(m)+" is completed")

    # con = sqlite3.connect("./data/real_trade.db")
    #sil_trade.to_sql('lent', con, if_exists='append', index=False)
    #con.close()    

26110, 13 is completed
26140, 58 is completed
26170, 39 is completed
26200, 79 is completed
26230, 375 is completed
26260, 181 is completed
26290, 163 is completed
26320, 214 is completed
26350, 294 is completed
26380, 260 is completed
26410, 215 is completed
26440, 58 is completed
26470, 135 is completed
26500, 127 is completed
26530, 159 is completed
26710, 128 is completed
11110, 53 is completed
11140, 50 is completed
11170, 71 is completed
11200, 134 is completed
11215, 74 is completed
11230, 174 is completed
11260, 150 is completed
11290, 155 is completed
11305, 58 is completed
11320, 159 is completed
11350, 347 is completed
11380, 120 is completed
11410, 115 is completed
11440, 146 is completed
11470, 153 is completed
11500, 199 is completed
11530, 206 is completed
11545, 82 is completed
11560, 160 is completed
11590, 139 is completed
11620, 166 is completed
11650, 177 is completed
11680, 209 is completed
11710, 273 is completed
11740, 173 is completed
*201701 is completed
26110,

26380, 284 is completed
26410, 239 is completed
26440, 74 is completed
26470, 116 is completed
26500, 146 is completed
26530, 180 is completed
26710, 116 is completed
11110, 56 is completed
11140, 102 is completed
11170, 144 is completed
11200, 231 is completed
11215, 186 is completed
11230, 259 is completed
11260, 226 is completed
11290, 403 is completed
11305, 149 is completed
11320, 227 is completed
11350, 520 is completed
11380, 189 is completed
11410, 249 is completed
11440, 274 is completed
11470, 286 is completed
11500, 297 is completed
11530, 362 is completed
11545, 98 is completed
11560, 340 is completed
11590, 266 is completed
11620, 251 is completed
11650, 377 is completed
11680, 475 is completed
11710, 596 is completed
11740, 295 is completed
*201709 is completed
26110, 21 is completed
26140, 49 is completed
26170, 43 is completed
26200, 87 is completed
26230, 285 is completed
26260, 180 is completed
26290, 205 is completed
26320, 277 is completed
26350, 245 is completed
26

11170, 118 is completed
11200, 107 is completed
11215, 60 is completed
11230, 152 is completed
11260, 214 is completed
11290, 310 is completed
11305, 137 is completed
11320, 221 is completed
11350, 452 is completed
11380, 249 is completed
11410, 157 is completed
11440, 148 is completed
11470, 262 is completed
11500, 328 is completed
11530, 312 is completed
11545, 95 is completed
11560, 207 is completed
11590, 172 is completed
11620, 206 is completed
11650, 163 is completed
11680, 137 is completed
11710, 160 is completed
11740, 187 is completed
*201805 is completed
26110, 13 is completed
26140, 51 is completed
26170, 37 is completed
26200, 53 is completed
26230, 244 is completed
26260, 138 is completed
26290, 172 is completed
26320, 178 is completed
26350, 198 is completed
26380, 221 is completed
26410, 92 is completed
26440, 47 is completed
26470, 102 is completed
26500, 94 is completed
26530, 140 is completed
26710, 83 is completed
11110, 55 is completed
11140, 82 is completed
11170, 

In [5]:
print(sil_trade.shape)
sil_trade.head()

(242886, 11)


,build_y,year,month,day,dong,apt_nm,size,jibun,ji_code,floor,real_price
0,2006,2017,1,1~10,영주동,우남이채롬,59.5862,5,26110,7,"21,000"
1,1998,2017,1,1~10,영주동,동아(92-0),59.57,92,26110,4,"15,700"
2,2016,2017,1,1~10,대창동1가,선경뉴빌 APT,63.6958,54-43,26110,7,"23,525"
3,1972,2017,1,11~20,대창동1가,부원,43.97,54-67,26110,6,"6,250"
4,1972,2017,1,11~20,대창동1가,부원,56.2,54-67,26110,8,"6,500"


In [6]:
#pip install openpyxl
#sil_trade.to_excel('./sil_trade_df1.xls')
sil_trade.to_csv('./sil_trade_df2.csv', encoding='ms949')

In [24]:
collect_AptTrade('201712','11680')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc

import platform

plt.rcParams['axes.unicode_minus']= False
if platform.system()== 'Windows':
    path= 'c:/Windows/Fonts/malgun.ttf'
    font_name= font_manager.FontProperties(fname=path).get_name()
    rc('font', family= font_name)
elif platform.system()=='Darwin':
    rc('font', family= 'AppleGothic')
else:
    print('포트를 지원하지 않습니다.')


In [8]:
test_raw = pd.read_csv('./dacon_data/test.csv', encoding='utf-8')
print(test_raw.shape)
test_raw.head()

(3918, 25)


,key,apartment_id,city,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,...,lowest_building_in_sites,heat_type,heat_fuel,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price
0,462533,3751,1,200912,21~31,1984,83.58,14,37.519926,127.052515,...,15.0,district,cogeneration,6648,107.19,2,3.0,1.0,corridor,0
1,764018,14029,1,201304,1~10,1968,107.17,10,37.572215,126.987643,...,8.0,individual,gas,46184,107.17,10,0.0,0.0,corridor,0
2,813528,12304,1,201309,21~30,1968,36.17,6,37.570067,127.010640,...,4.0,individual,gas,45540,39.66,58,2.0,1.0,stairway,0
3,845097,22241,1,201312,1~10,2007,45.67,8,37.556938,126.859255,...,15.0,individual,gas,19069,87.11,1,3.0,2.0,stairway,0
4,856338,316,1,201401,1~10,1988,41.85,4,37.652256,127.082003,...,5.0,individual,gas,704,56.70,90,2.0,1.0,stairway,0


In [9]:
TA = pd.read_csv('./AptTrade_2017_2018_2.csv', encoding='ms949', low_memory=False)
print(TA.shape)
TA.tail()

(242888, 11)


,build_y,year,month,day,dong,apt_nm,size,jibun,ji_code,floor,real_price
242883,2009,2018,10,1~10,강일동,강일리버파크10단지,114.81,695,11740,10,"84,800"
242884,2009,2018,10,1~10,강일동,강일리버파크10단지,84.53,695,11740,7,"72,000"
242885,2009,2018,10,11~20,강일동,강일리버파크6단지,84.74,684,11740,13,"75,000"
242886,없음,없음,없음,없음,없음,없음,없음,없음,없음,없음,없음
242887,복수,복수,복수,복수,복수,복수,복수,복수,복수,복수,복수


In [10]:
TS= test_raw[11:].copy()
print(TS.shape)
TS.head()

(3907, 25)


,key,apartment_id,city,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,...,lowest_building_in_sites,heat_type,heat_fuel,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price
11,1369751,3581,0,201702,1~10,1993,84.8700,5,35.069846,128.978010,...,18.0,individual,gas,30471,105.38,76,3.0,2.0,stairway,0
12,1379723,11908,1,201703,1~10,2005,181.5100,5,37.515753,127.062764,...,6.0,individual,gas,45171,205.27,48,4.0,2.0,stairway,0
13,1389544,26553,0,201704,11~20,1984,64.5200,3,35.146249,129.008208,...,5.0,individual,-,58482,71.72,20,3.0,1.0,stairway,0
14,1394472,36531,0,201704,1~10,2017,84.9972,23,35.183156,129.113972,...,29.0,individual,NaN,150241,112.49,240,3.0,2.0,stairway,0
15,1395869,9344,0,201704,21~30,1985,76.5000,3,35.187516,129.068324,...,5.0,individual,gas,40469,90.18,15,3.0,1.0,stairway,0


In [11]:
year_list=[]
month_list=[]

for dt1 in TS['transaction_year_month']:
    dt1= str(dt1)
    year_list.append(int(dt1[:4]))
    month_list.append(int(dt1[4:]))

TS['transaction_year'] = pd.Series(year_list, index= TS.index)
TS['transaction_month'] = pd.Series(month_list, index= TS.index)    

In [22]:
gu_list= []
for i in TS['address_by_law'].values:
    gu_list.append(str(i)[:5])

TS['region'] = pd.Series(gu_list, index= TS.index)

In [12]:
TS.isnull().any()

key                                   False
apartment_id                          False
city                                  False
transaction_year_month                False
transaction_date                      False
year_of_completion                    False
exclusive_use_area                    False
floor                                 False
latitude                              False
longitude                             False
address_by_law                        False
total_parking_capacity_in_site         True
total_household_count_in_sites        False
apartment_building_count_in_sites     False
tallest_building_in_sites              True
lowest_building_in_sites               True
heat_type                              True
heat_fuel                              True
room_id                               False
supply_area                           False
total_household_count_of_area_type    False
room_count                             True
bathroom_count                  

In [23]:
TS[['transaction_year_month','transaction_year','transaction_month','address_by_law', 'region']].head(10)

,transaction_year_month,transaction_year,transaction_month,address_by_law,region
11,201702,2017,2,2638010500,26380
12,201703,2017,3,1168010500,11680
13,201704,2017,4,2653010600,26530
14,201704,2017,4,2647010200,26470
15,201704,2017,4,2647010100,26470
16,201705,2017,5,2629010700,26290
17,201705,2017,5,2653010200,26530
18,201706,2017,6,2641010900,26410
19,201706,2017,6,2635010500,26350
20,201707,2017,7,1117012900,11170


In [26]:
#TS.iloc[[0]]
len(TS['transaction_year'].values)

3907

In [82]:
sil_df= pd.DataFrame()

In [83]:
# for i in TR_city0['address_by_law'].values:
#     idx= adr_code[adr_code['법정동코드']== i].index
#     dong_name_list.append(adr_code.loc[idx, '법정동명'].values)    
# 'transaction_year','transaction_month', 'exclusive_use_area', 'year_of_completion', 'floor'

for i in range(len(TS['transaction_year'].values)):
    
    ke= TS['key'].values[i]
    ye= TS['transaction_year'].values[i]
    mo= TS['transaction_month'].values[i]
    ar= TS['exclusive_use_area'].values[i]
    cy= TS['year_of_completion'].values[i]
    fl= abs(TS['floor'].values[i])
    gu= TS['region'].values[i]
    da= TS['transaction_date'].values[i]

#     query_str= "년 == "+ str(ye) + "and 월 == "+str(mo) + " and 전용면적 =="+str(ar) + "and 건축년도 == "+str(cy) +" and 층 =="+str(fl)
#     TA0= TA.query(query_str)

    TA0= TA.loc[(TA['year'] == str(ye)) & (TA['month'] == str(mo)) & (TA['build_y'] == str(cy)) 
                & (TA['floor'] == str(fl)) & (TA['size'] == str(ar)) & (TA['ji_code'] == str(gu)) & (TA['day'] == da)]    

#     TA0= TA.loc[(TA['year'] == str(ye)) & (TA['month'] == str(mo)) & (TA['build_y'] == str(cy)) 
#                 & (TA['floor'] == str(fl)) & (TA['ji_code'] == str(gu))]        

    if len(TA0)== 0:
        print('쿼리를 찾지못했습니다. key: ', ke, 'step:', i)
        TA0= pd.DataFrame(TA.iloc[-2]).T
    elif len(TA0) > 1:
        print('복수개의 쿼리을 찾았습니다. key: ', ke, 'step:', i)
        TA0= pd.DataFrame(TA.iloc[-1]).T
    else:
        print('쿼리 삽입완료 key:', ke, 'step:', i)

    sil_df = pd.concat([sil_df,TA0])


쿼리 삽입완료 key: 1369751 step: 0
쿼리 삽입완료 key: 1379723 step: 1
쿼리 삽입완료 key: 1389544 step: 2
쿼리 삽입완료 key: 1394472 step: 3
쿼리 삽입완료 key: 1395869 step: 4
쿼리 삽입완료 key: 1411847 step: 5
쿼리 삽입완료 key: 1419442 step: 6
쿼리 삽입완료 key: 1435592 step: 7
쿼리 삽입완료 key: 1436429 step: 8
쿼리 삽입완료 key: 1438962 step: 9
쿼리 삽입완료 key: 1444406 step: 10
쿼리 삽입완료 key: 1445883 step: 11
쿼리 삽입완료 key: 1450094 step: 12
쿼리 삽입완료 key: 1452779 step: 13
쿼리 삽입완료 key: 1458536 step: 14
쿼리 삽입완료 key: 1459610 step: 15
쿼리 삽입완료 key: 1460129 step: 16
쿼리 삽입완료 key: 1462023 step: 17
쿼리 삽입완료 key: 1462947 step: 18
쿼리 삽입완료 key: 1463225 step: 19
쿼리 삽입완료 key: 1463595 step: 20
쿼리 삽입완료 key: 1473795 step: 21
쿼리를 찾지못했습니다. key:  1474416 step: 22
쿼리 삽입완료 key: 1480440 step: 23
쿼리 삽입완료 key: 1482449 step: 24
쿼리 삽입완료 key: 1484643 step: 25
쿼리 삽입완료 key: 1484823 step: 26
쿼리 삽입완료 key: 1488114 step: 27
쿼리 삽입완료 key: 1488682 step: 28
쿼리 삽입완료 key: 1489423 step: 29
쿼리 삽입완료 key: 1489927 step: 30
쿼리 삽입완료 key: 1491157 step: 31
쿼리 삽입완료 key: 1492227 step: 32
쿼리 삽입완료 key: 1

쿼리 삽입완료 key: 1561274 step: 266
쿼리 삽입완료 key: 1561310 step: 267
쿼리 삽입완료 key: 1561355 step: 268
쿼리 삽입완료 key: 1561366 step: 269
쿼리 삽입완료 key: 1561385 step: 270
쿼리 삽입완료 key: 1561393 step: 271
쿼리 삽입완료 key: 1561446 step: 272
쿼리 삽입완료 key: 1561517 step: 273
복수개의 쿼리을 찾았습니다. key:  1561784 step: 274
쿼리 삽입완료 key: 1562077 step: 275
쿼리 삽입완료 key: 1562098 step: 276
쿼리를 찾지못했습니다. key:  1562135 step: 277
쿼리 삽입완료 key: 1562215 step: 278
쿼리 삽입완료 key: 1562342 step: 279
쿼리를 찾지못했습니다. key:  1562468 step: 280
쿼리 삽입완료 key: 1562611 step: 281
쿼리 삽입완료 key: 1562681 step: 282
쿼리 삽입완료 key: 1562694 step: 283
쿼리 삽입완료 key: 1562846 step: 284
쿼리 삽입완료 key: 1562851 step: 285
쿼리 삽입완료 key: 1562870 step: 286
쿼리 삽입완료 key: 1562959 step: 287
쿼리 삽입완료 key: 1563044 step: 288
쿼리 삽입완료 key: 1563116 step: 289
쿼리 삽입완료 key: 1563147 step: 290
쿼리 삽입완료 key: 1563429 step: 291
쿼리 삽입완료 key: 1563437 step: 292
쿼리 삽입완료 key: 1563862 step: 293
쿼리 삽입완료 key: 1564056 step: 294
쿼리 삽입완료 key: 1564249 step: 295
쿼리 삽입완료 key: 1564250 step: 296
쿼리 삽입완료 key: 15643

쿼리 삽입완료 key: 1577259 step: 530
쿼리 삽입완료 key: 1577320 step: 531
쿼리 삽입완료 key: 1577440 step: 532
쿼리 삽입완료 key: 1577522 step: 533
쿼리 삽입완료 key: 1577525 step: 534
쿼리 삽입완료 key: 1577625 step: 535
쿼리 삽입완료 key: 1577673 step: 536
쿼리 삽입완료 key: 1577674 step: 537
쿼리 삽입완료 key: 1577675 step: 538
쿼리 삽입완료 key: 1577707 step: 539
쿼리 삽입완료 key: 1577743 step: 540
쿼리를 찾지못했습니다. key:  1577760 step: 541
쿼리 삽입완료 key: 1577799 step: 542
쿼리 삽입완료 key: 1577835 step: 543
쿼리 삽입완료 key: 1577856 step: 544
쿼리를 찾지못했습니다. key:  1577913 step: 545
쿼리 삽입완료 key: 1577945 step: 546
쿼리 삽입완료 key: 1577979 step: 547
쿼리 삽입완료 key: 1578055 step: 548
쿼리 삽입완료 key: 1578057 step: 549
쿼리 삽입완료 key: 1578062 step: 550
쿼리 삽입완료 key: 1578165 step: 551
쿼리 삽입완료 key: 1578170 step: 552
쿼리 삽입완료 key: 1578282 step: 553
쿼리 삽입완료 key: 1578316 step: 554
쿼리 삽입완료 key: 1578317 step: 555
쿼리 삽입완료 key: 1578335 step: 556
쿼리 삽입완료 key: 1578395 step: 557
쿼리 삽입완료 key: 1578400 step: 558
쿼리 삽입완료 key: 1578410 step: 559
쿼리 삽입완료 key: 1578474 step: 560
쿼리 삽입완료 key: 1578535 step: 

쿼리 삽입완료 key: 1587159 step: 794
쿼리 삽입완료 key: 1587197 step: 795
쿼리 삽입완료 key: 1587213 step: 796
쿼리 삽입완료 key: 1587216 step: 797
쿼리 삽입완료 key: 1587246 step: 798
쿼리 삽입완료 key: 1587247 step: 799
쿼리 삽입완료 key: 1587293 step: 800
쿼리 삽입완료 key: 1587297 step: 801
쿼리 삽입완료 key: 1587318 step: 802
쿼리 삽입완료 key: 1587340 step: 803
쿼리 삽입완료 key: 1587366 step: 804
쿼리 삽입완료 key: 1587368 step: 805
쿼리 삽입완료 key: 1587386 step: 806
쿼리 삽입완료 key: 1587458 step: 807
쿼리 삽입완료 key: 1587479 step: 808
쿼리 삽입완료 key: 1587480 step: 809
쿼리 삽입완료 key: 1587507 step: 810
쿼리 삽입완료 key: 1587523 step: 811
쿼리 삽입완료 key: 1587524 step: 812
쿼리 삽입완료 key: 1587539 step: 813
쿼리 삽입완료 key: 1587553 step: 814
쿼리 삽입완료 key: 1587581 step: 815
쿼리 삽입완료 key: 1587615 step: 816
쿼리 삽입완료 key: 1587617 step: 817
쿼리 삽입완료 key: 1587627 step: 818
쿼리 삽입완료 key: 1587629 step: 819
쿼리 삽입완료 key: 1587650 step: 820
쿼리 삽입완료 key: 1587654 step: 821
쿼리 삽입완료 key: 1587664 step: 822
쿼리 삽입완료 key: 1587665 step: 823
쿼리 삽입완료 key: 1587722 step: 824
쿼리 삽입완료 key: 1587772 step: 825
쿼리 삽입완료 

쿼리 삽입완료 key: 1592278 step: 1054
쿼리 삽입완료 key: 1592297 step: 1055
쿼리를 찾지못했습니다. key:  1592339 step: 1056
쿼리 삽입완료 key: 1592346 step: 1057
쿼리 삽입완료 key: 1592379 step: 1058
쿼리 삽입완료 key: 1592383 step: 1059
쿼리 삽입완료 key: 1592412 step: 1060
쿼리 삽입완료 key: 1592425 step: 1061
쿼리를 찾지못했습니다. key:  1592430 step: 1062
쿼리 삽입완료 key: 1592453 step: 1063
쿼리 삽입완료 key: 1592460 step: 1064
쿼리 삽입완료 key: 1592469 step: 1065
쿼리 삽입완료 key: 1592471 step: 1066
쿼리 삽입완료 key: 1592480 step: 1067
쿼리 삽입완료 key: 1592481 step: 1068
쿼리 삽입완료 key: 1592488 step: 1069
쿼리 삽입완료 key: 1592492 step: 1070
쿼리 삽입완료 key: 1592504 step: 1071
쿼리 삽입완료 key: 1592509 step: 1072
쿼리 삽입완료 key: 1592545 step: 1073
쿼리 삽입완료 key: 1592550 step: 1074
쿼리 삽입완료 key: 1592581 step: 1075
쿼리 삽입완료 key: 1592625 step: 1076
쿼리 삽입완료 key: 1592645 step: 1077
쿼리 삽입완료 key: 1592647 step: 1078
쿼리 삽입완료 key: 1592653 step: 1079
쿼리 삽입완료 key: 1592657 step: 1080
쿼리 삽입완료 key: 1592673 step: 1081
쿼리 삽입완료 key: 1592675 step: 1082
쿼리 삽입완료 key: 1592689 step: 1083
쿼리 삽입완료 key: 1592691 step: 1

...

쿼리 삽입완료 key: 1603411 step: 2830
쿼리 삽입완료 key: 1603415 step: 2831
복수개의 쿼리을 찾았습니다. key:  1603416 step: 2832
복수개의 쿼리을 찾았습니다. key:  1603420 step: 2833
쿼리 삽입완료 key: 1603423 step: 2834
쿼리 삽입완료 key: 1603424 step: 2835
쿼리 삽입완료 key: 1603425 step: 2836
쿼리 삽입완료 key: 1603426 step: 2837
쿼리 삽입완료 key: 1603427 step: 2838
쿼리 삽입완료 key: 1603428 step: 2839
쿼리 삽입완료 key: 1603430 step: 2840
쿼리 삽입완료 key: 1603431 step: 2841
쿼리 삽입완료 key: 1603434 step: 2842
쿼리 삽입완료 key: 1603439 step: 2843
쿼리 삽입완료 key: 1603440 step: 2844
쿼리 삽입완료 key: 1603442 step: 2845
쿼리 삽입완료 key: 1603444 step: 2846
쿼리 삽입완료 key: 1603445 step: 2847
쿼리 삽입완료 key: 1603446 step: 2848
쿼리 삽입완료 key: 1603447 step: 2849
쿼리 삽입완료 key: 1603449 step: 2850
쿼리 삽입완료 key: 1603450 step: 2851
쿼리 삽입완료 key: 1603453 step: 2852
쿼리 삽입완료 key: 1603454 step: 2853
쿼리 삽입완료 key: 1603455 step: 2854
쿼리 삽입완료 key: 1603456 step: 2855
쿼리 삽입완료 key: 1603458 step: 2856
복수개의 쿼리을 찾았습니다. key:  1603459 step: 2857
쿼리 삽입완료 key: 1603460 step: 2858
쿼리를 찾지못했습니다. key:  1603461 step: 2859
쿼리 삽입완료

쿼리를 찾지못했습니다. key:  1603954 step: 3082
쿼리 삽입완료 key: 1603958 step: 3083
쿼리 삽입완료 key: 1603960 step: 3084
쿼리 삽입완료 key: 1603961 step: 3085
쿼리 삽입완료 key: 1603963 step: 3086
쿼리 삽입완료 key: 1603964 step: 3087
쿼리 삽입완료 key: 1603965 step: 3088
쿼리 삽입완료 key: 1603968 step: 3089
쿼리 삽입완료 key: 1603973 step: 3090
쿼리를 찾지못했습니다. key:  1603977 step: 3091
쿼리 삽입완료 key: 1603978 step: 3092
쿼리 삽입완료 key: 1603979 step: 3093
쿼리 삽입완료 key: 1603984 step: 3094
쿼리 삽입완료 key: 1603988 step: 3095
쿼리 삽입완료 key: 1603989 step: 3096
쿼리 삽입완료 key: 1603991 step: 3097
쿼리 삽입완료 key: 1603992 step: 3098
쿼리 삽입완료 key: 1603993 step: 3099
쿼리 삽입완료 key: 1603994 step: 3100
쿼리 삽입완료 key: 1603998 step: 3101
쿼리 삽입완료 key: 1603999 step: 3102
쿼리 삽입완료 key: 1604001 step: 3103
복수개의 쿼리을 찾았습니다. key:  1604012 step: 3104
쿼리 삽입완료 key: 1604013 step: 3105
쿼리 삽입완료 key: 1604015 step: 3106
쿼리 삽입완료 key: 1604016 step: 3107
쿼리 삽입완료 key: 1604017 step: 3108
쿼리 삽입완료 key: 1604021 step: 3109
쿼리 삽입완료 key: 1604022 step: 3110
쿼리 삽입완료 key: 1604026 step: 3111
쿼리 삽입완료 key: 160402

쿼리 삽입완료 key: 1604510 step: 3336
쿼리 삽입완료 key: 1604513 step: 3337
쿼리 삽입완료 key: 1604518 step: 3338
쿼리 삽입완료 key: 1604519 step: 3339
쿼리 삽입완료 key: 1604520 step: 3340
쿼리 삽입완료 key: 1604521 step: 3341
쿼리 삽입완료 key: 1604522 step: 3342
쿼리 삽입완료 key: 1604525 step: 3343
쿼리 삽입완료 key: 1604526 step: 3344
쿼리 삽입완료 key: 1604528 step: 3345
쿼리 삽입완료 key: 1604530 step: 3346
쿼리 삽입완료 key: 1604532 step: 3347
쿼리 삽입완료 key: 1604538 step: 3348
쿼리 삽입완료 key: 1604539 step: 3349
쿼리 삽입완료 key: 1604542 step: 3350
쿼리 삽입완료 key: 1604544 step: 3351
쿼리 삽입완료 key: 1604546 step: 3352
쿼리 삽입완료 key: 1604548 step: 3353
쿼리 삽입완료 key: 1604550 step: 3354
쿼리 삽입완료 key: 1604551 step: 3355
쿼리 삽입완료 key: 1604552 step: 3356
쿼리 삽입완료 key: 1604554 step: 3357
쿼리 삽입완료 key: 1604556 step: 3358
쿼리 삽입완료 key: 1604557 step: 3359
쿼리 삽입완료 key: 1604558 step: 3360
쿼리 삽입완료 key: 1604560 step: 3361
쿼리 삽입완료 key: 1604561 step: 3362
쿼리 삽입완료 key: 1604562 step: 3363
쿼리 삽입완료 key: 1604563 step: 3364
쿼리 삽입완료 key: 1604564 step: 3365
쿼리 삽입완료 key: 1604566 step: 3366
쿼리 삽입완료 

쿼리 삽입완료 key: 1604928 step: 3592
쿼리 삽입완료 key: 1604929 step: 3593
쿼리 삽입완료 key: 1604930 step: 3594
쿼리 삽입완료 key: 1604935 step: 3595
쿼리 삽입완료 key: 1604936 step: 3596
쿼리 삽입완료 key: 1604937 step: 3597
쿼리 삽입완료 key: 1604938 step: 3598
쿼리 삽입완료 key: 1604940 step: 3599
쿼리 삽입완료 key: 1604944 step: 3600
쿼리 삽입완료 key: 1604947 step: 3601
쿼리 삽입완료 key: 1604949 step: 3602
쿼리 삽입완료 key: 1604950 step: 3603
쿼리 삽입완료 key: 1604951 step: 3604
쿼리 삽입완료 key: 1604952 step: 3605
쿼리 삽입완료 key: 1604954 step: 3606
쿼리 삽입완료 key: 1604955 step: 3607
쿼리 삽입완료 key: 1604956 step: 3608
쿼리 삽입완료 key: 1604957 step: 3609
쿼리 삽입완료 key: 1604958 step: 3610
쿼리 삽입완료 key: 1604959 step: 3611
쿼리 삽입완료 key: 1604960 step: 3612
쿼리를 찾지못했습니다. key:  1604961 step: 3613
쿼리 삽입완료 key: 1604963 step: 3614
쿼리 삽입완료 key: 1604964 step: 3615
쿼리 삽입완료 key: 1604965 step: 3616
쿼리 삽입완료 key: 1604966 step: 3617
쿼리 삽입완료 key: 1604969 step: 3618
쿼리를 찾지못했습니다. key:  1604970 step: 3619
쿼리 삽입완료 key: 1604971 step: 3620
쿼리 삽입완료 key: 1604972 step: 3621
쿼리 삽입완료 key: 1604974 step: 3

쿼리 삽입완료 key: 1605295 step: 3846
복수개의 쿼리을 찾았습니다. key:  1605296 step: 3847
쿼리 삽입완료 key: 1605301 step: 3848
쿼리 삽입완료 key: 1605302 step: 3849
쿼리 삽입완료 key: 1605303 step: 3850
쿼리 삽입완료 key: 1605304 step: 3851
쿼리 삽입완료 key: 1605305 step: 3852
쿼리 삽입완료 key: 1605308 step: 3853
쿼리 삽입완료 key: 1605309 step: 3854
쿼리 삽입완료 key: 1605310 step: 3855
쿼리 삽입완료 key: 1605312 step: 3856
쿼리를 찾지못했습니다. key:  1605313 step: 3857
쿼리 삽입완료 key: 1605314 step: 3858
쿼리 삽입완료 key: 1605315 step: 3859
쿼리 삽입완료 key: 1605316 step: 3860
쿼리 삽입완료 key: 1605317 step: 3861
쿼리 삽입완료 key: 1605319 step: 3862
쿼리 삽입완료 key: 1605321 step: 3863
복수개의 쿼리을 찾았습니다. key:  1605322 step: 3864
쿼리 삽입완료 key: 1605323 step: 3865
쿼리 삽입완료 key: 1605324 step: 3866
쿼리 삽입완료 key: 1605325 step: 3867
쿼리 삽입완료 key: 1605327 step: 3868
쿼리 삽입완료 key: 1605328 step: 3869
쿼리 삽입완료 key: 1605329 step: 3870
쿼리 삽입완료 key: 1605330 step: 3871
쿼리 삽입완료 key: 1605331 step: 3872
쿼리 삽입완료 key: 1605332 step: 3873
쿼리 삽입완료 key: 1605335 step: 3874
쿼리 삽입완료 key: 1605337 step: 3875
쿼리 삽입완료 key: 160

In [84]:
sil_df.shape

(3907, 11)

In [85]:
pr_list=[]
for j in sil_df['real_price'].values:
    if j == '없음':
        pr_list.append(0)
    elif j == '복수':
        pr_list.append(1)
    else:
        b = j.split(',')
        c = ''
        for i in range(len(b)):
            c= c + b[i]
        pr_list.append(int(c)*10000)  
    

In [86]:
#del sil_df['거래금액 변환']
sil_df['real_price2']= pd.Series(pr_list, index= sil_df.index)

In [88]:
sil_df.to_csv('./find_df3.csv', encoding='ms949')

In [178]:
TA.columns
TA.shape

(242888, 11)

In [197]:
    i= 2185
    
    ke= TS['key'].values[i]
    ye= TS['transaction_year'].values[i]
    mo= TS['transaction_month'].values[i]
    ar= TS['exclusive_use_area'].values[i]
    cy= TS['year_of_completion'].values[i]
    fl= abs(TS['floor'].values[i])
    gu= TS['region'].values[i]
    da= TS['transaction_date'].values[i]


    TA0= TA.loc[(TA['year'] == str(ye)) & (TA['month'] == str(mo)) & (TA['build_y'] == str(cy)) 
                & (TA['floor'] == str(fl)) & (TA['size'] == str(ar)) & (TA['ji_code'] == str(gu)) & (TA['day'] == da)]       
    
    TA0

,build_y,year,month,day,dong,apt_nm,size,jibun,ji_code,floor,real_price
231182,2012,2018,9,1~10,광장동,광장힐스테이트,59.99,582,11215,6,"110,000"
